In [4]:
import pandas as pd 
from Bio import SeqIO
#handle fasta files
from Bio import SeqIO
#handle alignment
from Bio import AlignIO
from Bio.Align import AlignInfo
from collections import Counter 
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

import seaborn as sns
import re
from ete3 import Tree, PhyloTree

from scipy.sparse import lil_matrix
from scipy import sparse
from scipy.sparse.linalg import spsolve

from numpy.linalg import solve, norm

from numpy.random import rand

In [1]:
path_local = "U:/users/mamorel/Projet_Convergence/condor-analysis/"

## Create DRMs Dataset

In [2]:

DRMs_file_update = path_local+"data/HIV_africa/preprocessing/RT_DRMupdate2019.txt"
DRM_list = []
with open(DRMs_file_update) as f:
    for line in f:
        DRM_list.append(line.strip())

In [5]:
drm_associated_positions = list(re.findall(r'\d+[0-9]', ",".join(DRM_list)))
variants = [i[-1] for i in DRM_list]

drms_df = pd.DataFrame([DRM_list, drm_associated_positions, variants]).T
drms_df.columns =["ancposmut", "RT_position", "mut"]
drms_df["pastml_root"] = [i[0] for i in drms_df.ancposmut]
AA = list("ARNDCQEGHILKMFPSTWYV")

drms_df = drms_df[(drms_df.mut.isin(AA)) & (drms_df.RT_position.astype(int) <= 250)]

In [6]:
tips_file = path_local+"data/HIV_africa/align.noCRF.jphmm_outgroup.aa.fa"
Align_tips = AlignIO.read(open(tips_file),"fasta")
Tips_df = pd.DataFrame([list(rec.seq) for rec in Align_tips], index = [rec.id for rec in Align_tips])
A_tips = Tips_df.describe()
consensus_tips = list("".join(A_tips.loc["top"].values))
step = int("".join(consensus_tips).find("PISPIE"))-1

In [7]:
phenotype = path_local+"data/HIV_africa/id_phenotype.txt"
pheno = []
with open(phenotype) as f:
    for line in f:
        pheno.append(line.strip())

In [8]:
real_nb_seq = []
for i,j in zip(drms_df.RT_position, drms_df.mut):
    if int(i)+step <= 250:
        real_nb_seq.append(Counter(Tips_df[int(i)+step])[j])
    else: 
        real_nb_seq.append(0)

drms_df["real_nb_seq"] = real_nb_seq

len(drms_df[(drms_df.real_nb_seq >= 10)])

29

In [9]:
drms_df = drms_df[drms_df.real_nb_seq >= 10]

## Detections

In [10]:
all_substitutions = pd.read_csv(
        path_local
        + "results/HIV_africa/condor/tested_results.tsv",
        sep="\t",
    )

#### MODEL MISPECIFICATION

In [38]:
all_substitutions_JTT = pd.read_csv(
        path_local
        + "results/HIV_africa/condor/model_misspecification_JTT/tested_results.tsv",
        sep="\t",
    )

## ADD FADE

In [12]:
FADE = pd.read_csv(path_local+"results/HIV_africa/FADE/datamonkey_fade_conjunction_HIVb.csv", sep=",")

FADE.columns = ["mut","position","rate", "bias", "proba", "BF", "composition", "substitutions"]
FADE["posmut"] = FADE["position"].astype(str)+FADE["mut"]

all_substitutions["posmut"] = [
"".join([str(i), j]) for i, j in zip(all_substitutions.position, all_substitutions.mut)
]

All_bayes_Fade = pd.merge(all_substitutions, FADE, on=["posmut", "position", "mut"], how = "left")


#### MODEL MISPECIFICATION

In [50]:
FADE_JTT = pd.read_csv(path_local+"results/HIV_africa/FADE/datamonkey_fade_conjunction_JTT.csv", sep=",")

FADE_JTT.columns = ["mut","position","rate", "bias", "proba", "BF", "composition", "substitutions"]
FADE_JTT["posmut"] = FADE_JTT["position"].astype(str)+FADE_JTT["mut"]

all_substitutions_JTT["posmut"] = [
"".join([str(i), j]) for i, j in zip(all_substitutions_JTT.position, all_substitutions_JTT.mut)
]

All_bayes_Fade_JTT = pd.merge(all_substitutions_JTT, FADE_JTT, on=["posmut", "position", "mut"], how = "left")


## ADD PCOC

In [51]:

pcoc_results_all = pd.read_csv(path_local+"results/HIV_africa/PCOC/align.noCRF.jphmm.aa.results.tsv", sep="\t")
subset_pcoc_results = pd.DataFrame(pcoc_results_all[pcoc_results_all.Sites.isin(all_substitutions.position)])
subset_pcoc_results.rename(columns={"Sites": "position"}, inplace = True)
All_bayes_fade_pcoc = pd.merge(All_bayes_Fade, subset_pcoc_results , on="position")

#### Model Misspecification

In [52]:

pcoc_results_all_JTT = pd.read_csv(path_local+"results/HIV_africa/PCOC/align.noCRF.jphmm.aa.results.tsv", sep="\t")
subset_pcoc_results_JTT = pd.DataFrame(pcoc_results_all[pcoc_results_all.Sites.isin(all_substitutions.position)])
subset_pcoc_results_JTT.rename(columns={"Sites": "position"}, inplace = True)
All_bayes_fade_pcoc_JTT = pd.merge(All_bayes_Fade_JTT, subset_pcoc_results_JTT , on="position")

In [53]:
print(len(All_bayes_fade_pcoc),len(All_bayes_fade_pcoc_JTT))

240 240


## Analysis

In [54]:
set(All_bayes_fade_pcoc[(All_bayes_fade_pcoc.PC > 0.8) & (All_bayes_fade_pcoc.correlation == "positive") ].position).intersection(drms_df.RT_position.values.astype(int))

{101, 138, 188, 210, 215, 219, 221, 227}

In [55]:
All_bayes_fade_pcoc.columns

Index(['pastml_root', 'consensus_roto', 'position', 'mut', 'max_anc',
       'ref_EEM', 'nbseq', 'evol_rate', 'genetic_distance',
       'substitution_rate', 'findability', 'type_substitution', 'details',
       'loss', 'loss_details', 'max_simu', 'variance', 'mean', 'pvalue_raw',
       'adjust_pvalue', 'adjust_pvalue_fdr', 'detected_EEM', 'posmut',
       'log-dep', 'log-indep', 'BF_x', 'correlation', 'rate', 'bias', 'proba',
       'BF_y', 'composition', 'substitutions', 'Indel_prop',
       'Indel_prop(ConvLeaves)', 'PCOC', 'PC', 'OC'],
      dtype='object')

In [56]:
methods_detect = []
#criterion bf : BF >=10 & c == "positive"
#criterion condor : p <= 0.1
#criterion pcoc : PC > 0.9
#criterion condorBT : p <= 0.1 & BF >=10 & c == "positive"

for p,bf,c,oc,pc, fade in  zip(All_bayes_fade_pcoc["adjust_pvalue"], All_bayes_fade_pcoc["BF_x"] , All_bayes_fade_pcoc["correlation"], All_bayes_fade_pcoc["OC"], All_bayes_fade_pcoc["PC"], All_bayes_fade_pcoc["BF_y"]) : 
    if (p <= 0.1) and (bf >= 20) and (c == "positive"):
        methods_detect.append("condorBT")
    elif (c == "positive") and (pc > 0.8):
        methods_detect.append("Pcoc_pos")
    elif (c == "positive") and (oc > 0.8):
        methods_detect.append("Pcoc_pos")
    elif (pc > 0.8):
        methods_detect.append("pcoc")
    elif (oc > 0.8):
        methods_detect.append("pcoc")
    elif fade > 0:
        methods_detect.append("fade")
    elif (bf >= 20) and (c == "positive"):
        methods_detect.append("BayesTraits")
    elif p <= 0.1:
        methods_detect.append("condor")
    else:
        methods_detect.append("none")

All_bayes_fade_pcoc["method"] = methods_detect

In [45]:
methods_detect = []
#criterion bf : BF >=10 & c == "positive"
#criterion condor : p <= 0.1
#criterion pcoc : PC > 0.9
#criterion condorBT : p <= 0.1 & BF >=10 & c == "positive"

for p,bf,c,oc,pc, fade in  zip(All_bayes_fade_pcoc_JTT["adjust_pvalue"], All_bayes_fade_pcoc_JTT["BF_x"] , All_bayes_fade_pcoc_JTT["correlation"], All_bayes_fade_pcoc_JTT["OC"], All_bayes_fade_pcoc_JTT["PC"], All_bayes_fade_pcoc_JTT["BF_y"]) : 
    if (p <= 0.1) and (bf >= 20) and (c == "positive"):
        methods_detect.append("condorBT")
    elif (c == "positive") and (pc > 0.8):
        methods_detect.append("Pcoc_pos")
    elif (c == "positive") and (oc > 0.8):
        methods_detect.append("Pcoc_pos")
    elif (pc > 0.8):
        methods_detect.append("pcoc")
    elif (oc > 0.8):
        methods_detect.append("pcoc")
    elif fade > 0:
        methods_detect.append("fade")
    elif (bf >= 20) and (c == "positive"):
        methods_detect.append("BayesTraits")
    elif p <= 0.1:
        methods_detect.append("condor")
    else:
        methods_detect.append("none")

All_bayes_fade_pcoc_JTT["method"] = methods_detect

In [57]:
risk = 0.1
classification = []
for det, anc,pos,mut in zip(All_bayes_fade_pcoc.adjust_pvalue, All_bayes_fade_pcoc.pastml_root, All_bayes_fade_pcoc.position, All_bayes_fade_pcoc.mut ):
    if det <= risk:
        if "".join([anc,str(pos),mut]) in drms_df["ancposmut"].values:
            classification.append("Detected DRM")
        else: 
            classification.append("Convergent candidate")
    else:
        if "".join([anc,str(pos),mut]) in drms_df["ancposmut"].values:
            classification.append("Not detected DRM")
        else: 
            classification.append("Random mutation")
All_bayes_fade_pcoc["classification"] = classification

In [59]:
risk = 0.1
classification = []
for det, anc,pos,mut in zip(All_bayes_fade_pcoc_JTT.adjust_pvalue, All_bayes_fade_pcoc_JTT.pastml_root, All_bayes_fade_pcoc_JTT.position, All_bayes_fade_pcoc_JTT.mut ):
    if det <= risk:
        if "".join([anc,str(pos),mut]) in drms_df["ancposmut"].values:
            classification.append("Detected DRM")
        else: 
            classification.append("Convergent candidate")
    else:
        if "".join([anc,str(pos),mut]) in drms_df["ancposmut"].values:
            classification.append("Not detected DRM")
        else: 
            classification.append("Random mutation")
All_bayes_fade_pcoc_JTT["classification"] = classification

In [60]:

TP = 0
FP = 0   

def table_2_condor(df, method):
    if method == "pc":
    #pc
        dataset = df[(df.PC >= 0.8) & (df.correlation == "positive") ]  
    elif method == "FADE":
    #FADE
        dataset = df[(df.BF_y >= 100)] 
    elif method == "FADE_inf":
        dataset = df[(df.BF_y > 1e16)] 
    #emergence
    elif method == "emergence":
        dataset = df[(df.adjust_pvalue <= 0.1) ]  
    elif method == "correlation":
    #correlation
        dataset = df[(df.BF_x >= 20) & (df.correlation == "positive")]
    #condor
    else: 
        dataset = df[(df.BF_x >= 20) & (df.correlation == "positive") & (df.adjust_pvalue <= 0.1)]  
    
    results = dict(Counter(dataset.classification))
    #TP = results["Detected DRM"]
    try : 
        TP = results["Detected DRM"]+ results["Not detected DRM"]
    except KeyError:
        TP = results["Detected DRM"]
    ALL = len(df)
    P = len(df[df.classification.isin(["Detected DRM", "Not detected DRM"])])
    N = len(df[~df.classification.isin(["Detected DRM", "Not detected DRM"])])
    PP = len(dataset)
    FN = P - TP
    FP = PP-TP
    TN = ALL-PP-FN
    Recall = TP/P
    Precision = TP/PP
    F1 = 2*(Precision*Recall)/(Precision+Recall)
    F1bis = TP/(TP+((1/2)*(FP+FN)))
    TPR = Recall
    TNR = TN/N
    bal_acc = (TPR + TNR)/2 
    print([np.round(i,2) for i in [TP, FP, FN, TN, Recall, Precision,  F1]])


for method in ["pc", "FADE", "FADE_inf", "emergence", "correlation", "condor"]:
    for df in [All_bayes_fade_pcoc, All_bayes_fade_pcoc_JTT]:
        print(method)
        table_2_condor(df, method)

pc
[10, 10, 19, 201, 0.34, 0.5, 0.41]
pc
[10, 10, 19, 201, 0.34, 0.5, 0.41]
FADE
[22, 66, 7, 145, 0.76, 0.25, 0.38]
FADE
[24, 69, 5, 142, 0.83, 0.26, 0.39]
FADE_inf
[13, 6, 16, 205, 0.45, 0.68, 0.54]
FADE_inf
[14, 8, 15, 203, 0.48, 0.64, 0.55]
emergence
[20, 67, 9, 144, 0.69, 0.23, 0.34]
emergence
[21, 78, 8, 133, 0.72, 0.21, 0.33]
correlation
[16, 3, 13, 208, 0.55, 0.84, 0.67]
correlation
[16, 2, 13, 209, 0.55, 0.89, 0.68]
condor
[15, 2, 14, 209, 0.52, 0.88, 0.65]
condor
[16, 1, 13, 210, 0.55, 0.94, 0.7]
